In [50]:
import pandas as pd
import numpy as np
import bisect
import scipy.stats as stats

In [95]:
df = pd.read_csv("https://raw.githubusercontent.com/lab-pep-itba/Clase-3---Clasificadores-Bayesianos/master/data/alturas-pesos-mils-train.csv", index_col="Unnamed: 0")
df_test = pd.read_csv("https://raw.githubusercontent.com/lab-pep-itba/Clase-3---Clasificadores-Bayesianos/master/data/alturas-pesos-mils-test.csv", index_col="Unnamed: 0")

In [3]:
df.head()

,Genero,Peso,Altura
7620,Mujer,61.235,162.402
1837,Hombre,97.432,181.908
3311,Hombre,73.324,172.459
9478,Mujer,55.193,157.748
7805,Mujer,56.886,151.798


In [4]:
df_mujeres = df[df['Genero']=='Mujer'].drop(columns=['Genero'])
df_hombres = df[df['Genero']=='Hombre'].drop(columns=['Genero'])
df.drop(columns='Genero', inplace=True)

In [5]:
p_mujer = len(df_mujeres)/len(df)
p_hombre = len(df_hombres)/len(df)
print(f'{p_mujer}+{p_hombre}={p_mujer+p_hombre}')

0.49975+0.50025=1.0


In [6]:
mu_p, mu_a = df.mean()
mu_p_m, mu_a_m = df_mujeres.mean()
mu_p_h, mu_a_h = df_hombres.mean()

In [12]:
def get_cov_mat_data(df):
    cov_mat = df.cov().to_numpy()
    std_x, std_y = np.sqrt(cov_mat.diagonal())
    std_xy = cov_mat[0][1]/std_x/std_y
    return std_x, std_y, std_xy
    
std_p, std_a, std_pa = get_cov_mat_data(df)
std_p_m, std_a_m, std_pa_m = get_cov_mat_data(df_mujeres)
std_p_h, std_a_h, std_pa_h = get_cov_mat_data(df_hombres)

d = pd.DataFrame([['Todos', mu_p, std_p, mu_a, std_a, std_pa],
              ['Mujeres', mu_p_m, std_p_m, mu_a_m, std_a_m, std_pa_m], 
              ['Hombres', mu_p_h, std_p_h, mu_a_h, std_a_h, std_pa_h]],
    columns=['Grupo', 'Media peso', 'Desvio peso', 'Media altura', 'Desvio altura', 'Correlacion'])

d

,Grupo,Media peso,Desvio peso,Media altura,Desvio altura,Correlacion
0,Todos,72.802476,14.624699,168.113424,9.829018,0.924378
1,Mujeres,61.137895,8.671726,161.304713,6.897148,0.848971
2,Hombres,84.455398,8.983019,174.915330,7.281646,0.860449


In [43]:
hist, pedges, aedges = np.histogram2d(df['Peso'], df['Altura'], bins=40)
hist_m, pedges_m, aedges_m = np.histogram2d(df_mujeres['Peso'], df_mujeres['Altura'], bins=40)
hist_h, pedges_h, aedges_h = np.histogram2d(df_hombres['Peso'], df_hombres['Altura'], bins=40)

hist /= len(df)
hist_m /= len(df_mujeres)
hist_h /= len(df_hombres)

In [14]:
hist

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 2., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [66]:
aciertos = 0

def get_bin(a, x):
    ind = bisect.bisect(a, x)
    if ind == len(a):
        ind -= 1
    return ind
    
    
for _, persona in df_test.iterrows():
    p_pa = hist[get_bin(pedges, persona['Peso'])][get_bin(aedges, persona['Altura'])]
    p_pa_h = hist_h[get_bin(pedges_h, persona['Peso'])][get_bin(aedges_h, persona['Altura'])]

#print(p_pa)
    if p_pa != 0:
        p_h = p_pa_h * p_hombre / p_pa
    else:
        p_h = np.random.rand()
        
    guess = 'Hombre' if p_h >0.5 else 'Mujer'
    #print(f"Adivine {'hombre' if p_h > 0.5 else 'mujer'} con p={p_h}")
    #print(f"Era {persona['Genero']}")
    if persona['Genero'] == guess:
        aciertos +=1
        
print(aciertos/len(df_test))


0.8435


In [59]:
def get_2dgaussian_dist(df):
    dist = stats.multivariate_normal(mean=df.mean(), cov=df.cov())
    return dist
    
pa, pa_m, pa_h = (get_2dgaussian_dist(dataframe) for dataframe in [df, df_mujeres, df_hombres])


In [77]:
def get_prob(dist, x, bin_size):
    return dist.cdf(x+bin_size/2)-dist.cdf(x-bin_size/2)

In [123]:
bin_size = np.array([1, 0.5]) # 1kg y 0.5cm

aciertos = 0
for _, persona in df.iterrows():
    measures = np.array([persona['Peso'], persona['Altura']])
    p_pa = get_prob(pa, measures, bin_size)
    p_pa_h = get_prob(pa_h, measures, bin_size)
    
    if p_pa != 0:
        p_h = p_pa_h * p_hombre / p_pa
    else:
        p_h = 1 if persona['Altura'] >= mu_a else 0
    
    guess = 'Hombre' if p_h>0.5 else 'Mujer'
    if persona['Genero'] == guess:
        aciertos +=1
        
print(aciertos/len(df))

0.854625
